In [1]:
# pytorch - 5

# Dataset & DataLoader

# mini-batch GD

# Dataset class : __init__() >> fetch data ; __len__() >> total rows ; __getitem__(index) >> get data at index

# DataLoader : dataset , batch_size , shuffle , num_workers , drop_last , collate_fn , sampler

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import Dataset,DataLoader

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder

In [3]:
# data loading
# breast cancer

url = 'https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/master/data.csv'
df = pd.read_csv(url)

df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [4]:
# dropping unwanted columns

df.drop(['id','Unnamed: 32'],axis=1,inplace=True)

In [5]:
# defining features & target

X = df.drop('diagnosis',axis=1)
y = df['diagnosis']

In [6]:
# train-test split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=38)

In [7]:
# scaling

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [8]:
# label encoding

le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

In [21]:
# to tensors

X_train_t = torch.from_numpy(X_train).float()
X_test_t = torch.from_numpy(X_test).float()

y_train_t = torch.from_numpy(y_train).float().unsqueeze(1)
y_test_t = torch.from_numpy(y_test).float().unsqueeze(1)

In [22]:
print(X_train_t.shape)
print(X_test_t.shape)

print(y_train_t.shape)
print(y_test_t.shape)

torch.Size([455, 30])
torch.Size([114, 30])
torch.Size([455, 1])
torch.Size([114, 1])


In [23]:
# custom Dataset class

class CustomDataset(Dataset):

  def __init__(self,feats,labels):
    self.feats = feats
    self.labels = labels

  def __len__(self):
    return len(self.feats)

  def __getitem__(self,idx):
    return self.feats[idx],self.labels[idx]

In [34]:
train_dataset = CustomDataset(X_train_t,y_train_t)
test_dataset = CustomDataset(X_test_t,y_test_t)

train_dataloader = DataLoader(train_dataset,batch_size=32,shuffle=True)
test_dataloader = DataLoader(test_dataset,batch_size=32,shuffle=False)

In [35]:
# defining simple NN

class SimpleNN(nn.Module):

  def __init__(self,n_feats):
    super().__init__()

    self.network = nn.Sequential(
        nn.Linear(n_feats,3),
        nn.ReLU(),
        nn.Linear(3,1),
        nn.Sigmoid()
    )

  def forward(self,feats):
    out = self.network(feats)
    return out

In [36]:
learning_rate = 0.1

epochs = 25

loss_fn = nn.BCELoss()

In [37]:
# training pipeline

model = SimpleNN(X_train_t.shape[1])

optimizer = optim.SGD(model.parameters(), lr=learning_rate)

for epoch in range(epochs):

  for batch_feats,batch_labels in train_dataloader:            # mini-batch training

    y_pred = model.forward(batch_feats)

    loss = loss_fn(y_pred,batch_labels)

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()

  print(f"Epoch: {epoch + 1} Loss: {loss.item()}")

Epoch: 1 Loss: 0.4142346978187561
Epoch: 2 Loss: 0.19739921391010284
Epoch: 3 Loss: 0.09792584925889969
Epoch: 4 Loss: 0.04300595074892044
Epoch: 5 Loss: 0.0455598421394825
Epoch: 6 Loss: 0.055622637271881104
Epoch: 7 Loss: 0.10331080108880997
Epoch: 8 Loss: 0.3401239216327667
Epoch: 9 Loss: 0.013666274957358837
Epoch: 10 Loss: 0.09049151837825775
Epoch: 11 Loss: 0.013942071236670017
Epoch: 12 Loss: 0.010882909409701824
Epoch: 13 Loss: 0.03287111967802048
Epoch: 14 Loss: 0.01387341134250164
Epoch: 15 Loss: 0.021374493837356567
Epoch: 16 Loss: 0.0496932752430439
Epoch: 17 Loss: 0.004515289328992367
Epoch: 18 Loss: 0.03447677940130234
Epoch: 19 Loss: 0.00021257184562273324
Epoch: 20 Loss: 0.01650269702076912
Epoch: 21 Loss: 0.020816372707486153
Epoch: 22 Loss: 0.012165271677076817
Epoch: 23 Loss: 0.005153955426067114
Epoch: 24 Loss: 0.004512532614171505
Epoch: 25 Loss: 0.04325132444500923


In [38]:
# metrics

# !pip install torchmetrics

from torchmetrics.classification import BinaryAccuracy

metrics = BinaryAccuracy()

In [45]:
# evaluation

model.eval()

with torch.no_grad():

    for batch_feats,batch_labels in test_dataloader:

        y_pred = model(batch_feats)

        metrics.update(y_pred,batch_labels)

accuracy = metrics.compute()

print(f"Accuracy: {accuracy:.2f}%")

Accuracy: 0.96%
